# [第10章: 機械翻訳](https://nlp100.github.io/ja/ch10.html)
本章では，日本語と英語の翻訳コーパスである[京都フリー翻訳タスク (KFTT)](http://www.phontron.com/kftt/index-ja.html)を用い，ニューラル機械翻訳モデルを構築する．ニューラル機械翻訳モデルの構築には，[fairseq](https://github.com/pytorch/fairseq)，[Hugging Face Transformers](https://github.com/huggingface/transformers)，[OpenNMT-py](https://github.com/OpenNMT/OpenNMT-py)などの既存のツールを活用せよ．

## [90. データの準備](https://nlp100.github.io/ja/ch10.html#90-%E3%83%87%E3%83%BC%E3%82%BF%E3%81%AE%E6%BA%96%E5%82%99)
機械翻訳のデータセットをダウンロードせよ．訓練データ，開発データ，評価データを整形し，必要に応じてトークン化などの前処理を行うこと．ただし，この段階ではトークンの単位として形態素（日本語）および単語（英語）を採用せよ．

In [1]:
ls Input/kftt-data-1.0/data/tok/*

Input/kftt-data-1.0/data/tok/kyoto-dev.en
Input/kftt-data-1.0/data/tok/kyoto-dev.ja
Input/kftt-data-1.0/data/tok/kyoto-test.en
Input/kftt-data-1.0/data/tok/kyoto-test.ja
Input/kftt-data-1.0/data/tok/kyoto-train.cln.en
Input/kftt-data-1.0/data/tok/kyoto-train.cln.ja


In [2]:
import torchtext
from tqdm import tqdm

In [3]:
field_ja = torchtext.data.Field(init_token="<sos>",
                                eos_token="<eos>")
field_en = torchtext.data.Field(init_token="<sos>",
                                eos_token="<eos>")
fields = [("ja", field_ja), ("en", field_en)]

In [4]:
def create_kftt_dataset(type_str: str):
    if type_str not in ["train.cln", "dev", "test"]:
        raise ValueError("Invalid type_str:%s" % type_str)

    path_ja = "Input/kftt-data-1.0/data/tok/kyoto-%s.ja" % type_str
    path_en = "Input/kftt-data-1.0/data/tok/kyoto-%s.en" % type_str
    corpus_ja, corpus_en = None, None

    with open(path_ja) as f_ja, open(path_en) as f_en:
        corpus_ja = [l_ja.replace("\n", "") for l_ja in tqdm(f_ja)]
        corpus_en = [l_en.replace("\n", "") for l_en in tqdm(f_en)]

    examples = [torchtext.data.Example.fromlist([ja, en], fields)
                for ja, en in zip(corpus_ja, corpus_en)]

    return torchtext.data.Dataset(examples, fields)

In [5]:
%%time
kftt_train = create_kftt_dataset("train.cln")
kftt_dev = create_kftt_dataset("dev")
kftt_test = create_kftt_dataset("test")

329882it [00:00, 731173.60it/s]
329882it [00:00, 934068.50it/s]
1166it [00:00, 407376.80it/s]
1166it [00:00, 469073.32it/s]
1160it [00:00, 431826.81it/s]
1160it [00:00, 414393.38it/s]

CPU times: user 5.1 s, sys: 620 ms, total: 5.72 s
Wall time: 5.65 s


In [6]:
%%time
field_ja.build_vocab(kftt_train, kftt_dev, kftt_test, min_freq=2)

CPU times: user 1.56 s, sys: 6.57 ms, total: 1.57 s
Wall time: 1.57 s


In [7]:
%%time
field_en.build_vocab(kftt_train, kftt_dev, kftt_test, min_freq=2)

CPU times: user 1.59 s, sys: 3.45 ms, total: 1.6 s
Wall time: 1.6 s


In [8]:
#Test
print([field_ja.vocab.stoi[word]
       for word in "日本 の 水墨 画 を 一変 さ せ た 。".split()])
print([field_en.vocab.stoi[word]
       for word in "He revolutionized the Japanese ink painting .".split()])

[42, 4, 5717, 550, 11, 10827, 25, 104, 10, 5]
[31, 0, 4, 60, 1551, 528, 6]


## [91. 機械翻訳モデルの訓練](https://nlp100.github.io/ja/ch10.html#91-%E6%A9%9F%E6%A2%B0%E7%BF%BB%E8%A8%B3%E3%83%A2%E3%83%87%E3%83%AB%E3%81%AE%E8%A8%93%E7%B7%B4)
90で準備したデータを用いて，ニューラル機械翻訳のモデルを学習せよ（ニューラルネットワークのモデルはTransformerやLSTMなど適当に選んでよい）．

## [92. 機械翻訳モデルの適用](https://nlp100.github.io/ja/ch10.html#92-%E6%A9%9F%E6%A2%B0%E7%BF%BB%E8%A8%B3%E3%83%A2%E3%83%87%E3%83%AB%E3%81%AE%E9%81%A9%E7%94%A8)
91で学習したニューラル機械翻訳モデルを用い，与えられた（任意の）日本語の文を英語に翻訳するプログラムを実装せよ．

## [93. BLEUスコアの計測](https://nlp100.github.io/ja/ch10.html#93-bleu%E3%82%B9%E3%82%B3%E3%82%A2%E3%81%AE%E8%A8%88%E6%B8%AC)
91で学習したニューラル機械翻訳モデルの品質を調べるため，評価データにおけるBLEUスコアを測定せよ．

## [94. ビーム探索](https://nlp100.github.io/ja/ch10.html#94-%E3%83%93%E3%83%BC%E3%83%A0%E6%8E%A2%E7%B4%A2)
91で学習したニューラル機械翻訳モデルで翻訳文をデコードする際に，ビーム探索を導入せよ．ビーム幅を1から100くらいまで適当に変化させながら，開発セット上のBLEUスコアの変化をプロットせよ．

## [95. サブワード化](https://nlp100.github.io/ja/ch10.html#95-%E3%82%B5%E3%83%96%E3%83%AF%E3%83%BC%E3%83%89%E5%8C%96)
トークンの単位を単語や形態素からサブワードに変更し，91-94の実験を再度実施せよ．

## [96. 学習過程の可視化](https://nlp100.github.io/ja/ch10.html#96-%E5%AD%A6%E7%BF%92%E9%81%8E%E7%A8%8B%E3%81%AE%E5%8F%AF%E8%A6%96%E5%8C%96)
[Tensorboard](https://www.tensorflow.org/tensorboard)などのツールを用い，ニューラル機械翻訳モデルが学習されていく過程を可視化せよ．可視化する項目としては，学習データにおける損失関数の値とBLEUスコア，開発データにおける損失関数の値とBLEUスコアなどを採用せよ．

## [97. ハイパー・パラメータの調整](https://nlp100.github.io/ja/ch10.html#97-%E3%83%8F%E3%82%A4%E3%83%91%E3%83%BC%E3%83%91%E3%83%A9%E3%83%A1%E3%83%BC%E3%82%BF%E3%81%AE%E8%AA%BF%E6%95%B4)
ニューラルネットワークのモデルや，そのハイパーパラメータを変更しつつ，開発データにおけるBLEUスコアが最大となるモデルとハイパーパラメータを求めよ．

## [98. ドメイン適応](https://nlp100.github.io/ja/ch10.html#98-%E3%83%89%E3%83%A1%E3%82%A4%E3%83%B3%E9%81%A9%E5%BF%9C)
[Japanese-English Subtitle Corpus (JESC)](https://nlp.stanford.edu/projects/jesc/index_ja.html)や[JParaCrawl](http://www.kecl.ntt.co.jp/icl/lirg/jparacrawl/)などの翻訳データを活用し，KFTTのテストデータの性能向上を試みよ．

## [99. 翻訳サーバの構築](https://nlp100.github.io/ja/ch10.html#99-%E7%BF%BB%E8%A8%B3%E3%82%B5%E3%83%BC%E3%83%90%E3%81%AE%E6%A7%8B%E7%AF%89)
ユーザが翻訳したい文を入力すると，その翻訳結果がウェブブラウザ上で表示されるデモシステムを構築せよ．